# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель. 

Практическая работа разделена на 2 части: 
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [2]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле: 
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке. 


In [3]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке 
from collections import Counter

def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    # TODO реализуйте функцию, которая подсчитывает все 1-gram 2-gram ... order-gram ngram'ы в тексте
    all_n_grams = []
    for sentence in train_text:
        words = sentence.split()
        if bos:
            words.insert(0, '<s>')
        if eos:
            words.append('</s>')
        
        for N in range(1,order+1):
            all_n_grams.extend([tuple(words[i: i + N]) for i in range(len(words) - N + 1)])
    ngrams = Counter(all_n_grams)  
    return ngrams

In [4]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1, 
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1, 
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2, 
        ('как',): 1, 
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }    
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6, 
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')
    
    
test_count_ngrams()  

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю. 

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [5]:
# функция подсчета вероятности через количество со сглаживанием Лапласа
def calculate_ngram_prob(ngram: Tuple[str], counts: Counter, V=None, k=0) -> float:
    # подсчитывет ngram со сглаживанием Лапласа
    # TODO
    
    ngram_count = counts[ngram]
    if V is None:
        unigram_counts = [c[1] for c in dict(counts).items() if len(c[0]) == 1]
        V = len(unigram_counts)
    
    if len(ngram) > 1:
        prefix_count = counts[ngram[:-1]]
        prob = (ngram_count + k)/(prefix_count + k*V)
    else:
        unigram_counts = [c[1] for c in dict(counts).items() if len(c[0]) == 1]
        prob = (ngram_count + k)/(sum(unigram_counts) + k*V)

    return prob

In [6]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")
    

test_calculate_ngram_prob()  

Test 1.b passed


Основной метрикой язковых моделей является перплексия. 

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [7]:
# Языковая модель 
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None
        
    @property
    def V(self) -> int:
        return len(self.vocab)
    
    def fit(self, train_text: List[str]) -> None:
        # TODO
        # Подсчет vocab и ngrams_count по обучающей выборке
        self.ngrams_count = count_ngrams(train_text, self.order, self.bos, self.eos)
        if self.vocab is None:
            self.vocab = [c[0] for c in dict(self.ngrams_count).items() if len(c[0]) == 1]
                
    
    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # TODO 
        # считаем логарифм вероятности конкретной нграмы
        prob = calculate_ngram_prob(ngram, self.ngrams_count, self.V, self.k)
        prob = np.log(prob)
        return prob
           
    def predict_log_proba(self, words: List[str]) -> float:
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        logprob = 0
        # TODO 
        
        for i in range(0,len(words)-self.order+1,1):
            # print(i,i+self.order)
            # print(tuple(words[i:i+self.order]))
            logprob += self.predict_ngram_log_proba(tuple(words[i:i+self.order]))
            
        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        return logprob
        
    def ppl(self, text: List[str]) -> float:
        #TODO 
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным, 
        #    не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        # 
            
        perplexity = 0
        words = text[0].split()
 
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']
        
        n = 0
        for i in range(0,len(words)-self.order+1,1):
            perplexity += self.predict_ngram_log_proba(tuple(words[i:i+self.order]))
            n+=1
            
        perplexity += self.predict_ngram_log_proba(tuple([words[0]]))
        n+=1
        
        perplexity = np.exp(-perplexity/n)
        return perplexity

In [8]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"]) 
    
    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"
    
    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (6 балла)

In [9]:
import random
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы. 
    # верните topk наиболее вероятных продолжений фразы prefix 
    
    prefix = tuple(prefix[0].split()[-lm.order+1:])
    
    probs = []
    zero_probs = []
    
    for word in lm.vocab:
        ngram = prefix + word
        if lm.ngrams_count[ngram]:
            probs.append((word,lm.predict_ngram_log_proba(ngram)))
        else:
            zero_probs.append((word,lm.predict_ngram_log_proba(ngram)))
    
    if len(probs) > 0:
        probs.sort(key=lambda x: x[1], reverse=True)
        
        if len(probs)<topk:
            probs = probs + random.sample(zero_probs, topk-len(probs))
    else:
        probs = random.sample(zero_probs,topk)
    return probs[:topk]
    

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?). 
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели. 

In [10]:
#Your code here     
train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
lm = NgramLM(order=3)
lm.fit(train_data)

In [11]:
predict_next_word(lm, ['сойду с'])

[(('ума',), np.float64(-1.4469189829363254)),
 (('времени',), np.float64(-2.833213344056216)),
 (('</s>',), np.float64(-2.833213344056216)),
 (('уйму',), np.float64(-2.833213344056216))]

In [12]:
import re
pattern = r'\[.*?\]'
pattern2 = r'/^[\p{L}\p{N}]+$/u'

text = []

with open('pushkin.txt', 'r', encoding='ANSI') as f_in:
    paragraph = []
    for line in f_in:
        if line.startswith("		"):
            line = line.strip()
            line = re.sub(pattern, '', line)
            line = re.sub(r'[^А-Яа-яёЁA-Za-z0-9- ]+', '', line)
            paragraph.append(line)
        elif len(line.strip())>0 and len(paragraph)>0 and line.upper() == line:
            text.append(" ".join(paragraph))
            paragraph = []

In [13]:
print(text[1])

Мой дядя самых честных правил Когда не в шутку занемог Он уважать себя заставил И лучше выдумать не мог Его пример другим наука Но боже мой какая скука С больным сидеть и день и ночь Не отходя ни шагу прочь Какое низкое коварство Полуживого забавлять Ему подушки поправлять Печально подносить лекарство Вздыхать и думать про себя Когда же черт возьмет тебя


In [14]:
len(text)

400

In [25]:
lm = NgramLM(order=3)
lm.fit(text)

In [26]:
lm.V

9734

In [27]:
import random
tokens_to_generate = 20
generate_to_end = False
is_random = True
tokens_generated = 0
poem = "У лукоморья дуб зеленый".split()


while tokens_generated < tokens_to_generate or generate_to_end:
    next_token_candidates = predict_next_word(lm, [" ".join(poem[-lm.order+1:])], topk=2)
    if is_random:
        next_token = random.choice(next_token_candidates)
    else:
        next_token = next_token_candidates[0]
    next_token = next_token[0][0]     
    
    if generate_to_end and next_token == '</s>':
        break
    
    if next_token[0].isupper():
        poem.append('\n')
        
    poem.append(next_token)
    tokens_generated += 1
    
print(" ".join(poem))

У лукоморья дуб зеленый 
 Луны при свете покорны бейся 
 Евгению негодуя ночи пугал заготовит уме брата 
 Опрятней доверчивую луна 
 Сбираясь рысью читать крови лани
